In [ ]:
!pip install flax jax jaxlib optax
!pip install ftfy regex tqdm
!pip install codecarbon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel, AutoProcessor
import jax
import jax.numpy as jnp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker(
    project_name="RS-ZSL-UNIVPM",
    measure_power_secs=15,
    output_file="emissions-testing.csv",
    log_level="debug",
    save_to_file=True,
)

tracker.start()

[codecarbon WARNING @ 09:54:15] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 09:54:15] [setup] RAM Tracking...
[codecarbon INFO @ 09:54:15] [setup] GPU Tracking...
[codecarbon INFO @ 09:54:15] No GPU found.
[codecarbon INFO @ 09:54:15] [setup] CPU Tracking...
[codecarbon DEBUG @ 09:54:15] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 09:54:15] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 09:54:15] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 09:54:15] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:54:16] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please co

In [ ]:
# Da modificare per cambiare modello
model_number = "FINAL2"

fine_tuned_model_path = "/content/drive/MyDrive/clip_tuned_model/CLIP_" + model_number
dataset_root = "/content/drive/My Drive/dataset-testing/"

print("Using model" + fine_tuned_model_path)

model = FlaxCLIPModel.from_pretrained(fine_tuned_model_path)
#model = FlaxCLIPModel.from_pretrained("flax-community/clip-rsicd-v2")
#model = FlaxCLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = AutoProcessor.from_pretrained("flax-community/clip-rsicd-v2")

label_names = ["port", "viaduct", "sparseresidential", "storagetanks", "stadium", "square", "school", "river", "resort", "railwaystation"]


text_prompts = [f"An aerial photo of a {label}" for label in label_names]
text_inputs = processor(text=text_prompts, return_tensors='np', padding=True)

print(label_names)
print(text_prompts)

Using model/content/drive/MyDrive/clip_tuned_model/CLIP_FINAL2


[codecarbon INFO @ 09:54:32] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 09:54:32] RAM : 4.75 W during 15.01 s [measurement time: 0.0031]
[codecarbon INFO @ 09:54:32] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 09:54:32] CPU : 42.50 W during 15.01 s [measurement time: 0.0031]
[codecarbon INFO @ 09:54:32] 0.000197 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 09:54:32] last_duration=15.005839347839355
------------------------
[codecarbon INFO @ 09:54:47] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 09:54:47] RAM : 4.75 W during 14.99 s [measurement time: 0.0142]
[codecarbon INFO @ 09:54:47] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 09:54:47] CPU : 42.50 W during 15.01 s [measurement time: 0.0029]
[codecarbon INFO @ 09:54:47] 0.000394 kWh of electricity used since the beginning.
[cod

preprocessor_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

['port', 'viaduct', 'sparseresidential', 'storagetanks', 'stadium', 'square', 'school', 'river', 'resort', 'railwaystation']
['An aerial photo of a port', 'An aerial photo of a viaduct', 'An aerial photo of a sparseresidential', 'An aerial photo of a storagetanks', 'An aerial photo of a stadium', 'An aerial photo of a square', 'An aerial photo of a school', 'An aerial photo of a river', 'An aerial photo of a resort', 'An aerial photo of a railwaystation']


In [ ]:
# Function to calculate top k accuracy
def calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=3):

    top_indices = jnp.argsort(-similarity_scores)[0][:k]


    ground_truth_index = label_names.index(ground_truth_label)

    top_k_accuracy = 1.0 if ground_truth_index in top_indices else 0.0

    return top_k_accuracy

In [ ]:

total_images = 0
total_top_1_accuracy = 0.0
total_top_3_accuracy = 0.0
total_top_5_accuracy = 0.0

# Load and preprocess images
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return processor(images=image, return_tensors='np')['pixel_values']

# Iterate through the dataset and perform inference
for image_name in os.listdir(dataset_root):
    image_path = os.path.join(dataset_root, image_name)
    image = load_image(image_path)

    image = jnp.array(image)

    image_features = model.get_image_features(pixel_values=image)
    text_features = model.get_text_features(**text_inputs)

    image_features = image_features / jnp.linalg.norm(image_features, axis=-1, keepdims=True)
    text_features = text_features / jnp.linalg.norm(text_features, axis=-1, keepdims=True)

    similarity_scores = jnp.dot(image_features, text_features.T)

    top_indices = np.argsort(-similarity_scores[0])[:3]
    top_labels = [(label_names[i], similarity_scores[0, i]) for i in top_indices]

    ground_truth_label = image_name.split('_')[0]

    top_1_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=1)
    top_3_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=3)
    top_5_accuracy = calculate_top_k_accuracy(similarity_scores, ground_truth_label, label_names, k=5)

    total_images += 1
    total_top_1_accuracy += top_1_accuracy
    total_top_3_accuracy += top_3_accuracy
    total_top_5_accuracy += top_5_accuracy

    mean_top_1_accuracy = total_top_1_accuracy / total_images
    mean_top_3_accuracy = total_top_3_accuracy / total_images
    mean_top_5_accuracy = total_top_5_accuracy / total_images


    print(f'Image: {image_path}')
    print(f'Ground Truth Label: {ground_truth_label}')
    for label, score in top_labels:
        print(f'{label}: {score:.4f}')
    print(f'Top-1 Accuracy: {mean_top_1_accuracy}')
    print(f'Top-3 Accuracy: {mean_top_3_accuracy}')
    print(f'Top-5 Accuracy: {mean_top_5_accuracy}')
    print()

Image: /content/drive/My Drive/dataset-testing/stadium_164.jpg
Ground Truth Label: stadium
stadium: 0.2797
square: 0.2369
school: 0.2347
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_269.jpg
Ground Truth Label: square
square: 0.2700
railwaystation: 0.2378
school: 0.2234
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0



[codecarbon INFO @ 09:55:02] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 09:55:02] RAM : 4.75 W during 14.96 s [measurement time: 0.0023]
[codecarbon INFO @ 09:55:02] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 09:55:02] CPU : 42.50 W during 14.97 s [measurement time: 0.0106]
[codecarbon INFO @ 09:55:02] 0.000591 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 09:55:02] last_duration=14.962608575820923
------------------------


Image: /content/drive/My Drive/dataset-testing/square_276.jpg
Ground Truth Label: square
square: 0.2458
river: 0.2263
viaduct: 0.2260
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_176.jpg
Ground Truth Label: stadium
stadium: 0.2744
school: 0.2570
square: 0.2487
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_299.jpg
Ground Truth Label: square
square: 0.2496
school: 0.2242
stadium: 0.2163
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_27.jpg
Ground Truth Label: square
square: 0.2713
railwaystation: 0.2383
school: 0.2311
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_70.jpg
Ground Truth Label: square
square: 0.2447
resort: 0.2287
school: 0.2134
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content

[codecarbon INFO @ 09:55:17] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 09:55:17] RAM : 4.75 W during 14.98 s [measurement time: 0.0031]
[codecarbon INFO @ 09:55:17] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 09:55:17] CPU : 42.50 W during 14.99 s [measurement time: 0.0017]
[codecarbon INFO @ 09:55:17] 0.000787 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 09:55:17] last_duration=14.976026773452759
------------------------


Image: /content/drive/My Drive/dataset-testing/square_328.jpg
Ground Truth Label: square
square: 0.2386
viaduct: 0.2275
river: 0.2197
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_234.jpg
Ground Truth Label: stadium
stadium: 0.2970
school: 0.2412
square: 0.2397
Top-1 Accuracy: 1.0
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_107.jpg
Ground Truth Label: stadium
school: 0.2520
stadium: 0.2491
square: 0.2305
Top-1 Accuracy: 0.9230769230769231
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/square_55.jpg
Ground Truth Label: square
square: 0.2811
school: 0.2357
resort: 0.2329
Top-1 Accuracy: 0.9285714285714286
Top-3 Accuracy: 1.0
Top-5 Accuracy: 1.0

Image: /content/drive/My Drive/dataset-testing/stadium_260.jpg
Ground Truth Label: stadium
stadium: 0.2969
school: 0.2827
square: 0.2566
Top-1 Accuracy: 0.9333333333333333
Top-3 Accurac

...

Image: /content/drive/My Drive/dataset-testing/port_348.jpg
Ground Truth Label: port
port: 0.2622
river: 0.2237
sparseresidential: 0.2201
Top-1 Accuracy: 0.7236286919831224
Top-3 Accuracy: 0.9201326100060278
Top-5 Accuracy: 0.9789029535864979

Image: /content/drive/My Drive/dataset-testing/port_324.jpg
Ground Truth Label: port
port: 0.2891
river: 0.2525
resort: 0.2239
Top-1 Accuracy: 0.7237119614341669
Top-3 Accuracy: 0.9201566736968967
Top-5 Accuracy: 0.9789093100331425

Image: /content/drive/My Drive/dataset-testing/port_217.jpg
Ground Truth Label: port
port: 0.2546
river: 0.2530
resort: 0.2350
Top-1 Accuracy: 0.7237951807228916
Top-3 Accuracy: 0.9201807228915663
Top-5 Accuracy: 0.9789156626506024

Image: /content/drive/My Drive/dataset-testing/railwaystation_236.jpg
Ground Truth Label: railwaystation
railwaystation: 0.2901
viaduct: 0.2899
river: 0.2527
Top-1 Accuracy: 0.7238783498946101
Top-3 Accuracy: 0.9202047576031316
Top-5 Accuracy: 0.9789220114423366

Image: /content/drive/My D

[codecarbon INFO @ 11:28:04] Energy consumed for RAM : 0.007414 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 11:28:04] RAM : 4.75 W during 14.94 s [measurement time: 0.0067]
[codecarbon INFO @ 11:28:04] Energy consumed for all CPUs : 0.066359 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:28:04] CPU : 42.50 W during 14.95 s [measurement time: 0.0016]
[codecarbon INFO @ 11:28:04] 0.073773 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:28:04] last_duration=14.935158729553223
------------------------


Image: /content/drive/My Drive/dataset-testing/port_18.jpg
Ground Truth Label: port
port: 0.2734
river: 0.2356
resort: 0.2276
Top-1 Accuracy: 0.7237751728283739
Top-3 Accuracy: 0.9203486624586715
Top-5 Accuracy: 0.9789600240456868

Image: /content/drive/My Drive/dataset-testing/railwaystation_16.jpg
Ground Truth Label: railwaystation
port: 0.2660
railwaystation: 0.2427
square: 0.2348
Top-1 Accuracy: 0.7235576923076923
Top-3 Accuracy: 0.9203725961538461
Top-5 Accuracy: 0.9789663461538461

Image: /content/drive/My Drive/dataset-testing/port_106.jpg
Ground Truth Label: port
river: 0.2538
port: 0.2488
viaduct: 0.2456
Top-1 Accuracy: 0.7233403424451788
Top-3 Accuracy: 0.9203965154701111
Top-5 Accuracy: 0.9789726644638029

Image: /content/drive/My Drive/dataset-testing/railwaystation_169.jpg
Ground Truth Label: railwaystation
railwaystation: 0.2805
port: 0.2540
viaduct: 0.2464
Top-1 Accuracy: 0.7234234234234235
Top-3 Accuracy: 0.9204204204204204
Top-5 Accuracy: 0.978978978978979

Image: /con

[codecarbon INFO @ 11:28:19] Energy consumed for RAM : 0.007434 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 11:28:19] RAM : 4.75 W during 14.96 s [measurement time: 0.0128]
[codecarbon INFO @ 11:28:19] Energy consumed for all CPUs : 0.066536 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:28:19] CPU : 42.50 W during 14.98 s [measurement time: 0.0065]
[codecarbon INFO @ 11:28:19] 0.073969 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:28:19] last_duration=14.9633207321167
------------------------


Image: /content/drive/My Drive/dataset-testing/port_9.jpg
Ground Truth Label: port
port: 0.2846
river: 0.2268
resort: 0.2220
Top-1 Accuracy: 0.7231045849565478
Top-3 Accuracy: 0.9202876835480971
Top-5 Accuracy: 0.9790230746179203

Image: /content/drive/My Drive/dataset-testing/port_75.jpg
Ground Truth Label: port
port: 0.2616
river: 0.2470
square: 0.2344
Top-1 Accuracy: 0.7231875374475734
Top-3 Accuracy: 0.920311563810665
Top-5 Accuracy: 0.9790293588975434

Image: /content/drive/My Drive/dataset-testing/railwaystation_142.jpg
Ground Truth Label: railwaystation
sparseresidential: 0.2420
railwaystation: 0.2319
school: 0.2300
Top-1 Accuracy: 0.7229709493860437
Top-3 Accuracy: 0.9203354297693921
Top-5 Accuracy: 0.9790356394129979

Image: /content/drive/My Drive/dataset-testing/port_268.jpg
Ground Truth Label: port
port: 0.2554
river: 0.2307
resort: 0.2171
Top-1 Accuracy: 0.7230538922155688
Top-3 Accuracy: 0.9203592814371258
Top-5 Accuracy: 0.9790419161676647



In [ ]:
print("Images processed:" + str(total_images))

Images processed:3340


In [ ]:
import pandas as pd
from datetime import datetime
import os

# Define function to save top-k scores to Excel
def save_scores(file_path, mean_top1, mean_top3, mean_top5):

    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")


    new_data = {'Date': [date], 'Time': [time], 'Top-1 Mean': [mean_top1], 'Top-3 Mean': [mean_top3], 'Top-5 Mean': [mean_top5], 'Model Number': [model_number]}
    df = pd.DataFrame(new_data)


    if os.path.exists(file_path):
        existing_df = pd.read_excel(file_path)
        df = pd.concat([existing_df, df], ignore_index=True)
    else:
        df.to_excel(file_path, index=False, sheet_name='Top-K Scores')

    df.to_excel(file_path, index=False, sheet_name='Top-K Scores')



excel_file_path = '/content/drive/MyDrive/punteggi.xlsx'


save_scores(excel_file_path, mean_top_1_accuracy, mean_top_3_accuracy, mean_top_5_accuracy)

print(f"Top-k scores saved to {excel_file_path}")

tracker.stop()

[codecarbon INFO @ 11:28:25] Energy consumed for RAM : 0.007442 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 11:28:25] RAM : 4.75 W during 6.18 s [measurement time: 0.0028]
[codecarbon INFO @ 11:28:25] Energy consumed for all CPUs : 0.066608 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:28:25] CPU : 42.50 W during 6.18 s [measurement time: 0.0062]
[codecarbon INFO @ 11:28:25] 0.074050 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:28:25] last_duration=6.175598382949829
------------------------
[codecarbon DEBUG @ 11:28:25] EmissionsData(timestamp='2024-06-19T11:28:25', project_name='RS-ZSL-UNIVPM', run_id='7b0ae3ab-5b49-4a37-bbee-660af2b7e156', duration=5648.109389543533, emissions=0.02114165133642484, emissions_rate=3.7431377259733736e-06, cpu_power=42.5, gpu_power=0.0, ram_power=4.753040313720703, cpu_energy=0.06660847032997343, gpu_energy=0, ram_energy=0.0074418782397146594, energy_consumed=0.07405034856968805, country_name='United States', co

Top-k scores saved to /content/drive/MyDrive/punteggi.xlsx


0.02114165133642484

In [ ]:
tracker.stop()

[codecarbon WARNING @ 11:28:25] Tracker already stopped !
[codecarbon INFO @ 11:28:25] Energy consumed for RAM : 0.007442 kWh. RAM Power : 4.753040313720703 W
[codecarbon DEBUG @ 11:28:25] RAM : 4.75 W during 0.04 s [measurement time: 0.0027]
[codecarbon INFO @ 11:28:25] Energy consumed for all CPUs : 0.066609 kWh. Total CPU Power : 42.5 W
[codecarbon DEBUG @ 11:28:25] CPU : 42.50 W during 0.05 s [measurement time: 0.0036]
[codecarbon INFO @ 11:28:25] 0.074051 kWh of electricity used since the beginning.
[codecarbon DEBUG @ 11:28:25] last_duration=0.044798851013183594
------------------------
[codecarbon DEBUG @ 11:28:25] EmissionsData(timestamp='2024-06-19T11:28:25', project_name='RS-ZSL-UNIVPM', run_id='7b0ae3ab-5b49-4a37-bbee-660af2b7e156', duration=5648.1666650772095, emissions=0.021141833972462682, emissions_rate=3.743132103941497e-06, cpu_power=42.5, gpu_power=0.0, ram_power=4.753040313720703, cpu_energy=0.06660905087126626, gpu_energy=0, ram_energy=0.007441937395957671, energy_c

0.021141833972462682